In [1]:
import numpy as np
import pandas as pd
import gc
import time
from pickle import dump
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
from imblearn.over_sampling import SMOTE 

In [3]:
# pip install imblearn

In [4]:
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.metrics import confusion_matrix

In [5]:
X=pd.read_csv('../input/data_process.csv', low_memory=False)

In [6]:
X.shape

(356251, 799)

In [7]:
X.head()

,Unnamed: 0,index,SK_ID_CURR,TARGET,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,...,CC_NAME_CONTRACT_STATUS_Signed_MAX,CC_NAME_CONTRACT_STATUS_Signed_MEAN,CC_NAME_CONTRACT_STATUS_Signed_SUM,CC_NAME_CONTRACT_STATUS_Signed_VAR,CC_NAME_CONTRACT_STATUS_nan_MIN,CC_NAME_CONTRACT_STATUS_nan_MAX,CC_NAME_CONTRACT_STATUS_nan_MEAN,CC_NAME_CONTRACT_STATUS_nan_SUM,CC_NAME_CONTRACT_STATUS_nan_VAR,CC_COUNT
0,0,0,100002,1.0,0,0,0,0,202500.0,406597.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,100003,0.0,1,0,1,0,270000.0,1293502.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,100004,0.0,0,1,0,0,67500.0,135000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,100006,0.0,1,0,0,0,135000.0,312682.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
4,4,4,100007,0.0,0,0,0,0,121500.0,513000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#df.columns.tolist()

# Préprocessing 

In [9]:
X = X[X['TARGET'].notnull()] 

In [10]:
X = X.drop("Unnamed: 0", axis=1)

In [11]:
X = X.drop("index", axis=1)

In [12]:
X = X.drop("SK_ID_CURR", axis=1)

In [13]:
Y = X["TARGET"]

In [14]:
X= X.drop("TARGET", axis=1)

## Gestion des valeurs manquantes

In [15]:
ratio_nas = X.isna().sum()/X.shape[0]

In [16]:
useful_columns_less_than_ratio = ratio_nas[ratio_nas<0.25].index

In [17]:
X = X[useful_columns_less_than_ratio]

In [18]:
X.shape

(307507, 507)

In [19]:
X[~np.isfinite(X)] = np.nan

In [20]:
imputer = SimpleImputer(strategy="median")

In [21]:
columns_keep = X.columns

In [22]:
X = imputer.fit_transform(X)

In [23]:
X = pd.DataFrame(X, columns=columns_keep)

## Gestion des outliers

In [24]:
# >>> from sklearn.ensemble import IsolationForest
# >>> X = [[-1.1], [0.3], [0.5], [100]]
# >>> clf = IsolationForest(random_state=0).fit(X)
# >>> clf.predict([[0.1], [0], [90]])

In [25]:
# numerics_columns=X.select_dtypes(include = ['int32','float64']).columns

In [26]:
# df_num=df[numerics_columns]

In [27]:
# df_num=df_num.sample(8)

In [28]:
# numeric_value = df_num.select_dtypes(include = ['int32','float64']).columns
# for column in numeric_value:
#     plt.figure(figsize = (10,2))
#     sns.set(font_scale=1)
#     sns.distplot(df_num[column], bins=50)
#     chaine = 'Distribution de : ' + column
#     plt.title(chaine)
#     plt.xlabel(column)
#     plt.show()

### Graphique via boxplot pour l'identification des points abérrants

In [29]:
# sns.set(font_scale=2)
# for column in numeric_value[1:]:
#     plt.figure(figsize = (9,7))
#     sns.boxplot(df_num[column])
#     chaine = 'Distribution de : ' + column
#     plt.title(chaine)
#     plt.xlabel(column)
#     plt.show()
# sns.set(font_scale=1)

In [31]:
# dict_quantile = {} 
# for res in set (X["AMT_INCOME_TOTAL"]): 
#     mask_res = X["AMT_INCOME_TOTAL"]=="25000" 
#     data_res=X.loc[mask_res]
#     if data_res.shape[0]>0: 
#         q1=X[numeric_columns].quantile(q=0.25)
#         q3=X[numeric_columns].quantile(q=0.75)
#         q2=X[numeric_columns].quantile(q=0.5)
#         IQR = q3-q1
#         borne_sup = q3 + 1.5*IQR 
#         borne_inf = q1 - 1.5*IQR 
#         dict_quantile[res]=(borne_inf, borne_sup, q2) 

In [35]:
def impute_outliers(row: pd.Series):
    ''' 
        # METHODE DE DOCSTRING AVEC TOUS LES COMMENTAIRES: return, etc
       elle permet d'imputer les valeurs abérantes par la moyenne lorsque els données sont 
       infé aux 1 quartilers et sup aux 3ème quantilles
       :param row : pandas Series
       :return the tranformated row
    '''
    for name_col_numerique in numeric_columns:
        category_pro=str(row["AMT_INCOME_TOTAL"])
        if category_pro in dict_quantile: 
            valeur_dico=dict_quantile[category_pro] 
            if row[name_col_numerique]<valeur_dico [0][name_col_numerique] or row [name_col_numerique] > valeur_dico[1][name_col_numerique]:
                row [name_col_numerique]=valeur_dico[2][name_col_numerique]
    return row 

In [36]:
# df2=df2.apply(impute_outliers, axis=1)

## Vérification s'il existe encore des outliers par matrice ou graphique

In [32]:
X.to_csv("X_clean.csv", index=False)
Y.to_csv("Y_clean.csv", index=False)